In [0]:
%skip
drop table aviation_project.gold.dim_date

In [0]:
%sql
Create or Replace table aviation_project.gold.dim_date as
SELECT distinct
    sha2(
        concat_ws('||', cast(YEAR as string), cast(MONTH as string), cast(DAY as string)),
        256
    ) AS DATE_SK,
    YEAR,
    MONTH,
    DAY
FROM aviation_project.silver.flights;


In [0]:
%sql
MERGE INTO aviation_project.gold.dim_date AS tgt
USING (
    SELECT DISTINCT
        YEAR,
        MONTH,
        DAY
    FROM aviation_project.silver.flights_stream
    WHERE YEAR IS NOT NULL
      AND MONTH IS NOT NULL
      AND DAY IS NOT NULL
) AS src
ON tgt.YEAR = src.YEAR
AND tgt.MONTH = src.MONTH
AND tgt.DAY = src.DAY

WHEN NOT MATCHED THEN
INSERT (
    DATE_SK,
    YEAR,
    MONTH,
    DAY
)
VALUES (
    sha2(
        concat_ws('||', cast(YEAR as string), cast(MONTH as string), cast(DAY as string)),
        256
    ),
    src.YEAR,
    src.MONTH,
    src.DAY
);


In [0]:
%sql

select * from aviation_project.gold.dim_date 

In [0]:
%sql

select * from aviation_project.gold.dim_date 

Data Quality check:
 - Data type
 - Null
 - primary key uniqueness

In [0]:
%sql
describe table aviation_project.gold.dim_date 

In [0]:
%sql
-- duplicate check
select count(date_sk) , date_sk from aviation_project.gold.dim_date 
group by date_sk
having count(date_sk) > 1

In [0]:
%sql
-- null values check
SELECT
  COUNT(*) AS total_records,

  SUM(CASE WHEN DATE_SK IS NULL THEN 1 ELSE 0 END) AS DATE_SK_NULLS,
  SUM(CASE WHEN YEAR IS NULL THEN 1 ELSE 0 END) AS YEAR_NULLS,
  SUM(CASE WHEN MONTH IS NULL THEN 1 ELSE 0 END) AS MONTH_NULLS,
  SUM(CASE WHEN DAY IS NULL THEN 1 ELSE 0 END) AS DAY_NULLS
FROM aviation_project.gold.dim_date;
